In [10]:
import pandas as pd
import yfinance as yf
import numpy as np
import datetime as dt
import copy
import matplotlib.pyplot as plt
from yahooquery import Ticker

In [11]:
#importando base
df_ibrx = pd.read_excel(r'C:\Users\drsan\OneDrive\Área de Trabalho\Python\ETFs_Onshore.xlsx',sheet_name = 'IBRX100')
df_ibrx.head()

,Código,Ação,Tipo,Qtde. Teórica,Part. (%),Setor,Subsetor,Segmento
0,ABEV3,AMBEV S/A,ON,4.354.228.928,2936.0,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes
1,ALPA4,ALPARGATAS,PN N1,187.007.704,328.0,Consumo Cíclico,"Tecidos, Vestuário e Calçados",Calçados
2,ALSO3,ALIANSCSONAE,ON NM,127.374.163,186.0,Financeiro,Exploração de Imóveis,Exploração de Imóveis
3,AZUL4,AZUL,PN N2,317.471.474,363.0,Bens Industriais,Transporte,Transporte Aéreo
4,B3SA3,B3,ON NM,2.046.021.644,6301.0,Financeiro,Serviços Financeiros Diversos,Serviços Financeiros Diversos


In [12]:
# dando aquele tapa
df = df_ibrx
df = pd.DataFrame(df) 
df = df.dropna()
df['Part. (%)'] /= 100000
df.rename(columns={'Código\xa0':'Codigo'}, inplace = True)
df['Codigo'] = df['Codigo'] + ".SA"
tickers = df['Codigo']

In [13]:
start = dt.date.today() + dt.timedelta(1) - dt.timedelta(59)
end = dt.date.today() + dt.timedelta(1)
prices = {}
for t in tickers:
    try:
        prices[t] = yf.download(t,start,end,interval='1d')  
    except: 
        pass
    prices[t].dropna(how='all', inplace = True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [16]:
prices2 = copy.deepcopy(prices)
for t in tickers:
    prices2[t]['roll_avg_5d'] = prices2[t]['Close'].rolling(5).mean()
    prices2[t]['roll_avg_10d'] = prices2[t]['Close'].rolling(10).mean()
    prices2[t]['roll_avg_20d'] = prices2[t]['Close'].rolling(20).mean()
    
    prices2[t]['roll_exp_5d'] = prices2[t]['Close'].ewm(span=5).mean()
    prices2[t]['roll_exp_10d'] = prices2[t]['Close'].ewm(span=10).mean()
    prices2[t]['roll_exp_20d'] = prices2[t]['Close'].ewm(span=20).mean()
    
    prices2[t]['roll_max_5d'] = prices2[t]['Close'].rolling(5).max()
    prices2[t]['roll_max_10d'] = prices2[t]['Close'].rolling(10).max()
    prices2[t]['roll_max_20d'] = prices2[t]['Close'].rolling(20).max()

    prices2[t]['roll_min_5d'] = prices2[t]['Close'].rolling(5).min()
    prices2[t]['roll_min_10d'] = prices2[t]['Close'].rolling(10).min()
    prices2[t]['roll_min_20d'] = prices2[t]['Close'].rolling(20).min()
    
    prices2[t].dropna(inplace=True)

In [25]:
prices3 = pd.DataFrame(columns=['ticker','Setor', 'Subsetor', 'Segmento', 'Close', 'Close_D-1', 'roll_avg_5d',
                                'roll_avg_20d', 'roll_exp_5d','roll_exp_20d','roll_max_5d','roll_max_20d','roll_min_5d','roll_min_20d'])
for t in range(len(tickers)):
    prices3.loc[t,'ticker'] = str(tickers[t]).replace('.SA','')
    prices3.iloc[t,1] = df[df['Codigo'] == tickers[t]]['Setor'].to_list()[0]
    prices3.iloc[t,2] = df[df['Codigo'] == tickers[t]]['Subsetor'].to_list()[0]
    prices3.iloc[t,3] = df[df['Codigo'] == tickers[t]]['Segmento'].to_list()[0]
    prices3.iloc[t,4] = prices2[tickers[t]]['Close'][-1]
    prices3.iloc[t,5] = prices2[tickers[t]]['Close'][-2]
    
    prices3.iloc[t,6] = prices2[tickers[t]]['roll_avg_5d'][-1]
    prices3.iloc[t,7] = prices2[tickers[t]]['roll_avg_20d'][-1]
    
    prices3.iloc[t,8] = prices2[tickers[t]]['roll_exp_5d'][-1]
    prices3.iloc[t,9] = prices2[tickers[t]]['roll_exp_20d'][-1]    

    prices3.iloc[t,10] = prices2[tickers[t]]['roll_max_5d'][-1]
    prices3.iloc[t,11] = prices2[tickers[t]]['roll_max_20d'][-1]    

    prices3.iloc[t,12] = prices2[tickers[t]]['roll_min_5d'][-1]
    prices3.iloc[t,13] = prices2[tickers[t]]['roll_min_20d'][-1]   
    
#variacoes
prices3['R1'] = prices3['Close'] / prices3['Close_D-1'] - 1
prices3['R2'] = prices3['Close'] / prices3['roll_exp_5d'] - 1
prices3['R3'] = prices3['Close'] / prices3['roll_avg_20d'] - 1
prices3['R4'] = (prices3['roll_max_20d'] + prices3['roll_min_20d']) / prices3['Close'] - 2

#ranking
prices3['Ranking_1'] = prices3['R1'].rank(ascending = True)
prices3['Ranking_2'] = prices3['R2'].rank(ascending = False)
prices3['Ranking_3'] = prices3['R3'].rank(ascending = True)
prices3['Ranking_4'] = prices3['R4'].rank(ascending= False)
prices3['Soma_total'] = prices3['Ranking_1'] + prices3['Ranking_2']*2 + prices3['Ranking_3']*2 + prices3['Ranking_4']
prices3['Ranking_Final'] = prices3['Soma_total'].rank(ascending= True)

#filtro
prices3[(prices3['Ranking_2']<=75)&(prices3['Ranking_3']<=75)].sort_values(['Ranking_Final'], ascending = True).head(10)

#prices3.sort_values(['Ranking_Final'], ascending = True).head(10)

,ticker,Setor,Subsetor,Segmento,Close,Close_D-1,roll_avg_5d,roll_avg_20d,roll_exp_5d,roll_exp_20d,...,R1,R2,R3,R4,Ranking_1,Ranking_2,Ranking_3,Ranking_4,Soma_total,Ranking_Final
60,LIGT3,Utilidade Pública,Energia Elétrica,Energia Elétrica,16.15,16.19,16.004,17.002,16.0643,16.7217,...,-0.00247072,0.00533483,-0.0501118,0.112694,38.0,49.0,10.0,14.0,170.0,1.0
72,PCAR3,Consumo não Cíclico,Comércio e Distribuição,Produtos de Limpeza,64.81,64.65,64.564,68.0015,64.7082,67.0635,...,0.0024748,0.00157257,-0.0469328,0.199352,52.0,61.0,11.0,2.0,198.0,2.0
55,ITUB4,Financeiro,Intermediários Financeiros,Bancos,24.48,24.05,24.202,25.585,24.3167,25.2065,...,0.0178794,0.00671742,-0.0431894,0.138072,81.0,45.0,16.0,9.0,212.0,6.0
41,ENGI11,Utilidade Pública,Energia Elétrica,Energia Elétrica,45.55,45.23,44.872,47.1705,45.113,46.3627,...,0.00707494,0.00968652,-0.0343541,0.0583974,60.0,36.0,25.0,31.0,213.0,7.0
23,CMIG4,Utilidade Pública,Energia Elétrica,Energia Elétrica,10.8,10.81,10.608,10.9917,10.6634,10.8737,...,-0.000925091,0.012806,-0.0174364,0.0452222,42.0,29.0,40.0,35.0,215.0,9.0
12,BPAN4,Financeiro,Intermediários Financeiros,Bancos,8.9,8.75,8.846,9.405,8.85748,9.21869,...,0.0171428,0.0048005,-0.0536949,0.0966293,80.0,51.0,9.0,19.0,219.0,13.0
7,BBDC4,Financeiro,Intermediários Financeiros,Bancos,21.17,20.7,20.83,21.795,20.9371,21.4983,...,0.0227053,0.0111242,-0.0286763,0.119509,88.0,33.0,28.0,11.0,221.0,14.5
19,BTOW3,Consumo Cíclico,Comércio,Produtos Diversos,115.14,116,114.148,118.928,114.691,117.06,...,-0.0074138,0.00391823,-0.0318553,0.0363036,23.0,53.0,26.0,40.0,221.0,14.5
70,NEOE3,Utilidade Pública,Energia Elétrica,Energia Elétrica,19.91,19.7,19.43,20.2885,19.5716,20.025,...,0.0106599,0.0172892,-0.0186559,0.034656,67.0,21.0,37.0,42.0,225.0,16.0
54,ITSA4,Financeiro,Intermediários Financeiros,Bancos,9.83,9.71,9.748,10.203,9.77836,10.0372,...,0.0123584,0.00528143,-0.0365579,0.111902,70.0,50.0,22.0,16.0,230.0,17.5


In [9]:
prices3.to_excel(r'C:\Users\drsan\OneDrive\Área de Trabalho\Python\IBRX100.xlsx',sheet_name = 'IBRX100')